# Linear Programming - Project
## CSCI 5654
***
**Name**: $<$Ketan Ramesh$>$ 
**Name**: $<$Shreyas Gopalakrishna$>$ 
***

## Vehicle Routing Problem

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import random
import pulp

In [104]:
numberOfCustomers = 5
capacityOfVehicle = 10
numberOfVehicles = 2
C = [i for i in range(1, numberOfCustomers+1)] #set of customers
V = [0] + C + [numberOfCustomers+1] #depot + customer nodes
demandOfCustomers = {i: np.random.randint(1, 10) for i in C}
demandOfCustomers[0] = 0
demandOfCustomers[numberOfCustomers+1] = 0

In [105]:
#   [0 1 2 3 0]
# 0 [i 1 1 1 0]
# 1 [1 i 1 1 1]
# 2 [1 1 i 1 1]
# 3 [1 1 1 i 1]
# 0 [0 1 1 1 i]

In [106]:
# Creating random coordinates
xCoordinates = np.random.rand(len(V))*1000
yCoordinates = np.random.rand(len(V))*1000

# Cost matrix
costMatrix = np.ndarray(shape=(len(V),len(V)))
for i in range(len(V)):
    for j in range(len(V)):
        if(i == 0 and j == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(j == 0 and i == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(i!=j):
            costMatrix[i][j] = distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]])
        else:
            costMatrix[i][j] = float('inf')
costMatrix

array([[         inf, 572.4659872 , 980.25034811, 322.5365231 ,
        157.87744866,  28.15904074,   0.        ],
       [572.4659872 ,          inf, 408.06786221, 328.67990884,
        414.59481105, 563.85920287, 337.25863798],
       [980.25034811, 408.06786221,          inf, 718.95928947,
        822.38811465, 970.83682024, 146.95318637],
       [322.5365231 , 328.67990884, 718.95928947,          inf,
        200.1331633 , 328.41773569, 665.20115834],
       [157.87744866, 414.59481105, 822.38811465, 200.1331633 ,
                 inf, 150.7935735 , 736.98618621],
       [ 28.15904074, 563.85920287, 970.83682024, 328.41773569,
        150.7935735 ,          inf, 880.33913709],
       [  0.        , 337.25863798, 146.95318637, 665.20115834,
        736.98618621, 880.33913709,          inf]])

In [107]:
len(costMatrix)

7

In [108]:
# PuLP class for vehicle routing

class CVRP:
    def __init__(self, numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix):
        self.numberOfCustomers = numberOfCustomers
        self.numberOfVehicles  = numberOfVehicles
        self.capacityOfVehicle = capacityOfVehicle
        self.demandOfCustomers = demandOfCustomers
        self.costMatrix        = costMatrix
        self.initializeLP()
    
    def initializeLP(self):
        self.cvrpLP = pulp.LpProblem("CVRP", pulp.LpMinimize)
        objective = None
        x,y = [], []
        constraint1 = None
        
        # objective function and variables
        for i in range(len(costMatrix)): #adding depot
            xTemp1 = []
            for j in range(len(costMatrix)):
                if(i != j):
                    xTemp2 = pulp.LpVariable('x('+str(i)+','+str(j)+')', cat='Binary')
                    xTemp1.append(xTemp2)
                    objective += xTemp2 * costMatrix[i][j]
                else:
                    xTemp1.append(None)
            x.append(xTemp1) 
        self.cvrpLP += objective
        
        for i in range(len(costMatrix)):
            y.append(pulp.LpVariable('y'+str(i), lowBound=0, cat='Continuous'))
        
        
        # constraints
        # ensure that all customers are visited exactly once
        for i in range(len(costMatrix)): #adding depot
            constraint1 = None
            for j in range(len(costMatrix)):
                if(i != j):
                    constraint1 += x[i][j]
            self.cvrpLP += constraint1 == 1
        
        # limits the maximum number of routes to the number of vehicles
        constraint2 = None
        for j in range(1, numberOfCustomers + 1): #not include depot
            constraint2 += x[0][j]
        self.cvrpLP += constraint2 <= self.numberOfVehicles
        
        # ensure together that the vehicle capacity is not exceeded
        for i in range(1, len(costMatrix) - 1):
            constarint3a, constarint3b  = None, None
            constarint3a = self.demandOfCustomers[i] <= y[i] 
            constarint3b = y[i] <= self.capacityOfVehicle
            self.cvrpLP += constarint3a
            self.cvrpLP += constarint3b
        
        # ensure together that the vehicle capacity is not exceeded
        for i in range(len(costMatrix)): #adding depot
            for j in range(len(costMatrix)):
                constraint4 = None
                if(i != j):
                    constraint4 += y[j] >= y[i] + self.demandOfCustomers[j]*x[i][j] - self.capacityOfVehicle*(1-x[i][j])
                    self.cvrpLP += constraint4
        
        #guarantee the correct flow of vehicles through the arcs, by stating that if a vehicle arrives to a node
        #then it must depart from this node
        for h in range(1, len(costMatrix) - 1):
            constraint5a, constraint5b = None, None
            for i in range(0, len(costMatrix) - 1):
                if(i != h):
                    constraint5a += x[i][h]
            for j in range(1, len(costMatrix)):
                if(j != h):
                    constraint5b += x[h][j]
            self.cvrpLP += constraint5a - constraint5b == 0
        
        print(self.cvrpLP)
        
    def solve(self):
        status = self.cvrpLP.solve()
        print(pulp.LpStatus[self.cvrpLP.status])
    
    def getResult(self):
        print("Objective value: ", pulp.value(self.cvrpLP.objective))
        for v in self.cvrpLP.variables():
            print(v.name, " = ", v.varValue)
        
        


In [109]:
lp = CVRP(numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix)

CVRP:
MINIMIZE
572.4659871992111*x(0,1) + 980.2503481112448*x(0,2) + 322.5365231002718*x(0,3) + 157.8774486614151*x(0,4) + 28.159040742515796*x(0,5) + 572.4659871992111*x(1,0) + 408.06786221489534*x(1,2) + 328.6799088355404*x(1,3) + 414.59481104626536*x(1,4) + 563.8592028661254*x(1,5) + 337.25863797780636*x(1,6) + 980.2503481112448*x(2,0) + 408.06786221489534*x(2,1) + 718.9592894700559*x(2,3) + 822.3881146536659*x(2,4) + 970.836820236137*x(2,5) + 146.95318636786195*x(2,6) + 322.5365231002718*x(3,0) + 328.6799088355404*x(3,1) + 718.9592894700559*x(3,2) + 200.13316330431607*x(3,4) + 328.4177356938102*x(3,5) + 665.2011583371147*x(3,6) + 157.8774486614151*x(4,0) + 414.59481104626536*x(4,1) + 822.3881146536659*x(4,2) + 200.13316330431607*x(4,3) + 150.79357349808444*x(4,5) + 736.9861862077528*x(4,6) + 28.159040742515796*x(5,0) + 563.8592028661254*x(5,1) + 970.836820236137*x(5,2) + 328.4177356938102*x(5,3) + 150.79357349808444*x(5,4) + 880.3391370914865*x(5,6) + 337.25863797780636*x(6,1) + 14

In [110]:
lp.solve()
lp.getResult()

Optimal
Objective value:  2061.2893478146584
x(0,1)  =  0.0
x(0,2)  =  0.0
x(0,3)  =  0.0
x(0,4)  =  0.0
x(0,5)  =  0.0
x(0,6)  =  1.0
x(1,0)  =  1.0
x(1,2)  =  0.0
x(1,3)  =  0.0
x(1,4)  =  0.0
x(1,5)  =  0.0
x(1,6)  =  0.0
x(2,0)  =  1.0
x(2,1)  =  0.0
x(2,3)  =  0.0
x(2,4)  =  0.0
x(2,5)  =  0.0
x(2,6)  =  0.0
x(3,0)  =  1.0
x(3,1)  =  0.0
x(3,2)  =  0.0
x(3,4)  =  0.0
x(3,5)  =  0.0
x(3,6)  =  0.0
x(4,0)  =  1.0
x(4,1)  =  0.0
x(4,2)  =  0.0
x(4,3)  =  0.0
x(4,5)  =  0.0
x(4,6)  =  0.0
x(5,0)  =  1.0
x(5,1)  =  0.0
x(5,2)  =  0.0
x(5,3)  =  0.0
x(5,4)  =  0.0
x(5,6)  =  0.0
x(6,0)  =  1.0
x(6,1)  =  0.0
x(6,2)  =  0.0
x(6,3)  =  0.0
x(6,4)  =  0.0
x(6,5)  =  0.0
y0  =  10.0
y1  =  4.0
y2  =  10.0
y3  =  7.0
y4  =  2.0
y5  =  8.0
y6  =  10.0
